In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from early_exit_resnet import *


In [2]:
block1 = HeadNetworkPart1(block=Bottleneck, in_planes=64, num_blocks=[3], num_classes=10)
block2 = HeadNetworkPart2(Bottleneck, 256, [4], num_classes=10)
block3 = HeadNetworkPart3(block=Bottleneck, in_planes=512, num_blocks=[6], num_classes=10)
block4 = TailNetwork(block=Bottleneck, in_planes=1024, num_blocks=[3, 4, 6, 3], num_classes=10)


In [4]:
combined_state_dict = torch.load('lightning_logs/version_10/checkpoints/epoch=11-step=780.ckpt')


/tmp/ipykernel_6736/3979702023.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  combined_state_dict = torch.load('lightning_logs/version_10/checkpoints/epoch=11-step=780.

In [5]:

block1_state_dict = {}
block2_state_dict = {}
block3_state_dict = {}
block4_state_dict = {}


for key, value in combined_state_dict['state_dict'].items():
    if key.startswith('head1'):
        block1_state_dict[key.removeprefix('head1.')] = value
    elif key.startswith('head2'):
        block2_state_dict[key.removeprefix('head2.')] = value
    elif key.startswith('head3'):
        block3_state_dict[key.removeprefix('head3.')] = value
    else:
        block4_state_dict[key.removeprefix('tail.')] = value


In [7]:
torch.save(block1_state_dict,"models/cifar10/block1.pth")
torch.save(block2_state_dict,"models/cifar10/block2.pth")
torch.save(block3_state_dict,"models/cifar10/block3.pth")
torch.save(block4_state_dict,"models/cifar10/block4.pth")

In [3]:
block1.load_state_dict(torch.load("models/cifar10/block1.pth"))
block2.load_state_dict(torch.load("models/cifar10/block2.pth"))
block3.load_state_dict(torch.load("models/cifar10/block3.pth"))
block4.load_state_dict(torch.load("models/cifar10/block4.pth"))

# head_net_part2.load_state_dict(torch.load("models/head2_resnet50.pth"))
# head_net_part3.load_state_dict(torch.load("models/head3_resnet50.pth"))
# tail_net.load_state_dict(torch.load("models/tail_resnet50.pth"))


/tmp/ipykernel_12467/3902660696.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block1.load_state_dict(torch.load("models/cifar10/block1.pth"))
/tmp/ipykernel_12467/3902

<All keys matched successfully>

In [10]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    total_samples = len(test_loader.dataset)
    correct = 0 # Assuming three exits

    with torch.no_grad():  # No need to compute gradients
        for inputs, labels in test_loader:
            # print(inputs)
            inputs = inputs.to(device)
            labels = labels.to(device)
            exit = model(inputs)[1]  # Forward pass
            softmax = torch.nn.Softmax(dim=1)
            exit_soft = softmax(exit)
            predictions = exit_soft.argmax(dim=1)
            # print(inputs.shape)
            correct += 1 if (predictions == labels) else 0

    accuracy = correct / total_samples
    return accuracy

In [4]:
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,transform=transform)
dataloader = DataLoader(test_set, batch_size=1, shuffle=True, num_workers=1)


Files already downloaded and verified


In [12]:
accuracy = evaluate_model(block1,dataloader, 'cpu')
print(accuracy)

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
t

KeyboardInterrupt: 

In [108]:
print(head_net_part1.conv1.weight)

Parameter containing:
tensor([[[[-0.1712,  0.1379, -0.0461],
          [-0.1524,  0.1505,  0.1423],
          [ 0.0199, -0.1704,  0.1499]],

         [[-0.1138,  0.0437, -0.1105],
          [ 0.1410,  0.0629, -0.1853],
          [-0.1650, -0.0616,  0.0838]],

         [[ 0.0817, -0.0064,  0.1190],
          [ 0.0860,  0.1002, -0.0764],
          [ 0.0427, -0.0219,  0.1117]]],


        [[[-0.0328,  0.0188, -0.0039],
          [-0.1313, -0.1366, -0.1489],
          [ 0.1330, -0.1349, -0.0988]],

         [[ 0.0967, -0.0589,  0.0178],
          [-0.0275,  0.0541, -0.0468],
          [ 0.1830,  0.0110,  0.1865]],

         [[-0.1955,  0.0400,  0.0822],
          [-0.1342,  0.1742, -0.1089],
          [ 0.1046,  0.0586,  0.1181]]],


        [[[ 0.0139, -0.0056, -0.1066],
          [-0.0118,  0.1503,  0.0908],
          [ 0.1657, -0.1041, -0.0535]],

         [[-0.0890, -0.1459, -0.1690],
          [-0.0583,  0.0189,  0.0926],
          [ 0.1022, -0.0586, -0.1416]],

         [[ 0.1612, -0

In [3]:
model = EarlyExitResNet50(num_classes=3)
model.load_state_dict(combined_state_dict['state_dict'])



<All keys matched successfully>

In [ ]:
from dataset import Flame2DataModule
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_module = Flame2DataModule(image_dir="../Flame2/254_RGB", batch_size=32,transform=transform)
data_module.setup()
print(data_module.evaluate_model(model,"cuda"))

In [5]:
from dataset import CacheDataset

In [6]:
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
custom_test_set = CacheDataset(test_set, models={'block1': block1, 'block2': block2, 'block3': block3, 'block4': block4},compute_logits=True)

Files already downloaded and verified
Computing and saving logits...


In [7]:
custom_test_set[0]

{'image': tensor([[[ 0.6338,  0.6531,  0.7694,  ...,  0.2267,  0.0134, -0.1804],
          [ 0.5174,  0.4981,  0.6531,  ...,  0.2073, -0.0060, -0.1223],
          [ 0.4981,  0.4981,  0.6338,  ...,  0.2654,  0.0910, -0.1029],
          ...,
          [-1.1109, -1.6149, -1.8281,  ..., -1.6924, -2.1771, -1.6537],
          [-1.2466, -1.4792, -1.7506,  ..., -1.9251, -1.8669, -2.0414],
          [-1.3823, -1.3435, -1.5567,  ..., -1.9638, -1.7700, -2.0220]],
 
         [[-0.2156, -0.2352, -0.1369,  ..., -0.5499, -0.6286, -0.7466],
          [-0.2156, -0.2549, -0.1762,  ..., -0.5499, -0.6286, -0.6876],
          [-0.2549, -0.2746, -0.2352,  ..., -0.4909, -0.5499, -0.6679],
          ...,
          [ 0.0204, -0.4516, -0.6876,  ..., -0.5106, -1.1596, -0.7466],
          [-0.1369, -0.4122, -0.7466,  ..., -0.8056, -0.8056, -1.1596],
          [-0.3139, -0.3532, -0.6679,  ..., -0.9039, -0.7662, -1.1006]],
 
         [[-1.2654, -1.3044, -1.2264,  ..., -1.5190, -1.5190, -1.5776],
          [-1.2264,